# Data Set -> Car Evaluation

Esse dataset é composto de 6 variáveis categóricas: compra, manutenção, portas, capacidade de pessoas, bagageiro e segurança.

A variável-alvo que indica a aceitação do carro:

unacc = Não aceitável

acc = Aceitável

good = Bom!

vgood = Muito Bom!

In [ ]:
# Dependências
#!pip install ucimlrepo scikit-learn pandas numpy

In [3]:
# importação do dataset
from ucimlrepo import fetch_ucirepo

# fetch dataset
car_evaluation = fetch_ucirepo(id=19)

# data (as pandas dataframes)
X = car_evaluation.data.features
y = car_evaluation.data.targets

# metadata
print(car_evaluation.metadata)

# variable information
print(car_evaluation.variables)

{'uci_id': 19, 'name': 'Car Evaluation', 'repository_url': 'https://archive.ics.uci.edu/dataset/19/car+evaluation', 'data_url': 'https://archive.ics.uci.edu/static/public/19/data.csv', 'abstract': 'Derived from simple hierarchical decision model, this database may be useful for testing constructive induction and structure discovery methods.', 'area': 'Other', 'tasks': ['Classification'], 'characteristics': ['Multivariate'], 'num_instances': 1728, 'num_features': 6, 'feature_types': ['Categorical'], 'demographics': [], 'target_col': ['class'], 'index_col': None, 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 1988, 'last_updated': 'Thu Aug 10 2023', 'dataset_doi': '10.24432/C5JP48', 'creators': ['Marko Bohanec'], 'intro_paper': {'ID': 249, 'type': 'NATIVE', 'title': 'Knowledge acquisition and explanation for multi-attribute decision making', 'authors': 'M. Bohanec, V. Rajkovič', 'venue': '8th Intl Workshop on Expert Systems and their Applications, 

In [4]:
print(X)

     buying  maint  doors persons lug_boot safety
0     vhigh  vhigh      2       2    small    low
1     vhigh  vhigh      2       2    small    med
2     vhigh  vhigh      2       2    small   high
3     vhigh  vhigh      2       2      med    low
4     vhigh  vhigh      2       2      med    med
...     ...    ...    ...     ...      ...    ...
1723    low    low  5more    more      med    med
1724    low    low  5more    more      med   high
1725    low    low  5more    more      big    low
1726    low    low  5more    more      big    med
1727    low    low  5more    more      big   high

[1728 rows x 6 columns]


In [5]:
print(y)

      class
0     unacc
1     unacc
2     unacc
3     unacc
4     unacc
...     ...
1723   good
1724  vgood
1725  unacc
1726   good
1727  vgood

[1728 rows x 1 columns]


In [6]:
y.value_counts()


class
unacc    1210
acc       384
good       69
vgood      65
Name: count, dtype: int64

# 1º PIPELINE

* Carrega o dataset Car Evaluation

* Faz one-hot encoding das variáveis categóricas

* Treina e avalia, com validação cruzada estratificada, os quatro modelos:

  1.   Árvore de Decisão
  2.   Bagging
  3.   AdaBoost (stumps por padrão)
  4.   Random Forest

* Reporta accuracy e F1 macro (métrica mais robusta a desbalanceamento)

* Separa um conjunto de teste para uma comparação final + matriz de confusão

In [8]:
# -*- coding: utf-8 -*-
%pip install scikit-learn
import numpy as np
import pandas as pd

from ucimlrepo import fetch_ucirepo

from sklearn.model_selection import train_test_split, StratifiedKFold, cross_validate
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, AdaBoostClassifier, RandomForestClassifier

from sklearn.metrics import classification_report, confusion_matrix

RANDOM_STATE = 42


  Using cached scikit_learn-1.7.2-cp313-cp313-win_amd64.whl.metadata (11 kB)
  Using cached scipy-1.16.2-cp313-cp313-win_amd64.whl.metadata (60 kB)
  Using cached joblib-1.5.2-py3-none-any.whl.metadata (5.6 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
Using cached scikit_learn-1.7.2-cp313-cp313-win_amd64.whl (8.7 MB)
Using cached joblib-1.5.2-py3-none-any.whl (308 kB)
Using cached scipy-1.16.2-cp313-cp313-win_amd64.whl (38.5 MB)
Using cached threadpoolctl-3.6.0-py3-none-any.whl (18 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:

# ---------------------------------------------------------------------
# 1) Dados
# ---------------------------------------------------------------------
car_evaluation = fetch_ucirepo(id=19)
X = car_evaluation.data.features.copy()
y = car_evaluation.data.targets.iloc[:, 0].copy()  # target vem como DF; pega a primeira coluna

# Todas as colunas de X são categóricas nesse dataset
categorical_cols = list(X.columns)


In [10]:
# ---------------------------------------------------------------------
# 2) Pré-processamento: OneHot para categóricas
# ---------------------------------------------------------------------
ohe = OneHotEncoder(handle_unknown="ignore")  # robusto a categorias raras no teste
preprocess = ColumnTransformer(
    transformers=[("cat", ohe, categorical_cols)],
    remainder="drop"
)


In [13]:
# ---------------------------------------------------------------------
# 3) Modelos
#    - Árvores com class_weight="balanced" por causa do desbalanceamento das classes
# ---------------------------------------------------------------------
tree_clf = DecisionTreeClassifier(
    criterion="gini",
    max_depth=None,
    min_samples_leaf=1,
    class_weight="balanced",
    random_state=RANDOM_STATE
)

bagging_clf = BaggingClassifier(
    estimator=DecisionTreeClassifier(
        criterion="gini",
        max_depth=None,
        min_samples_leaf=1,
        class_weight="balanced",
        random_state=RANDOM_STATE
    ),
    n_estimators=200,
    max_samples=1.0,
    max_features=1.0,
    bootstrap=True,
    n_jobs=-1,
    random_state=RANDOM_STATE
)

adaboost_clf = AdaBoostClassifier(
    # Por padrão o AdaBoost usa stumps (árvores de profundidade 1).
    # Você pode explicitar se quiser: estimator=DecisionTreeClassifier(max_depth=1, random_state=RANDOM_STATE)
    n_estimators=300,
    learning_rate=0.5,
    random_state=RANDOM_STATE
)

rf_clf = RandomForestClassifier(
    n_estimators=400,
    max_depth=None,
    min_samples_leaf=1,
    class_weight="balanced",
    n_jobs=-1,
    random_state=RANDOM_STATE
)

models = {
    "DecisionTree": tree_clf,
    "Bagging(Tree)": bagging_clf,
    "AdaBoost": adaboost_clf,
    "RandomForest": rf_clf
}


In [15]:
# ---------------------------------------------------------------------
# 4) Validação cruzada CV (k-fold estratificado) + holdout final
# ---------------------------------------------------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=RANDOM_STATE, stratify=y
)

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)

scoring = {
    "accuracy": "accuracy",
    "f1_macro": "f1_macro"
}

def summarize_scores(name, cv_results):
    acc = cv_results["test_accuracy"]
    f1m = cv_results["test_f1_macro"]
    return {
        "model": name,
        "cv_acc_mean": acc.mean(),
        "cv_acc_std": acc.std(ddof=1),
        "cv_f1m_mean": f1m.mean(),
        "cv_f1m_std": f1m.std(ddof=1),
    }

summary_rows = []
final_reports = {}

for name, clf in models.items():
    pipe = Pipeline(steps=[
        ("prep", preprocess),
        ("clf", clf)
    ])

    # Validação cruzada no conjunto de treino
    cv_results = cross_validate(
        pipe, X_train, y_train,
        scoring=scoring,
        cv=cv,
        n_jobs=-1,
        return_estimator=False
    )
    summary_rows.append(summarize_scores(name, cv_results))

    # Ajuste final e avaliação no holdout
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)

    final_reports[name] = {
        "classification_report": classification_report(y_test, y_pred, digits=4),
        "confusion_matrix": confusion_matrix(y_test, y_pred, labels=np.unique(y))
    }

c:\Users\PICHAU\Desktop\Faculdade\Mestrado\Mineração de Dados\Atividade 2\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\PICHAU\Desktop\Faculdade\Mestrado\Mineração de Dados\Atividade 2\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\PICHAU\Desktop\Faculdade\Mestrado\Mineração de Dados\Atividade 2\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted sampl

In [16]:
# ---------------------------------------------------------------------
# 5) Resultados
# ---------------------------------------------------------------------
summary_df = pd.DataFrame(summary_rows).sort_values("cv_f1m_mean", ascending=False)
pd.set_option("display.max_colwidth", None)

print("\n==================== RESUMO CV (5-fold) ====================")
print(summary_df.to_string(index=False, float_format=lambda x: f"{x:0.4f}"))

print("\n==================== TESTE (HOLDOUT 20%) ====================")
for name in models.keys():
    print(f"\n>>> {name}")
    print(final_reports[name]["classification_report"])
    print("Matriz de confusão (linhas = verdade, colunas = predito):")
    print(final_reports[name]["confusion_matrix"])


==================== RESUMO CV (5-fold) ====================
        model  cv_acc_mean  cv_acc_std  cv_f1m_mean  cv_f1m_std
Bagging(Tree)       0.9718      0.0119       0.9495      0.0136
 DecisionTree       0.9559      0.0113       0.9347      0.0160
 RandomForest       0.9522      0.0101       0.8830      0.0239
     AdaBoost       0.8502      0.0170       0.4803      0.0520

==================== TESTE (HOLDOUT 20%) ====================

>>> DecisionTree
              precision    recall  f1-score   support

         acc     0.8961    0.8961    0.8961        77
        good     0.8571    0.8571    0.8571        14
       unacc     0.9754    0.9835    0.9794       242
       vgood     1.0000    0.8462    0.9167        13

    accuracy                         0.9538       346
   macro avg     0.9322    0.8957    0.9123       346
weighted avg     0.9539    0.9538    0.9536       346

Matriz de confusão (linhas = verdade, colunas = predito):
[[ 69   2   6   0]
 [  2  12   0   0]
 [  4 

# 2º PIPELINE

* Carrega o dataset Car Evaluation

* Holdout (80/20) + validação cruzada estratificada (5-fold) no treino

* Compara Árvore, Bagging, AdaBoost e Random Forest usando F1 macro como métrica de seleção.

* Escolhe melhor algoritmo pela média do F1 macro na CV.

* Roda GridSearchCV apenas no melhor algoritmo.

* Compara desempenho **antes vs. depois** do tuning no conjunto de teste e exibe matriz de confusão.

In [17]:
# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd

from ucimlrepo import fetch_ucirepo

from sklearn.model_selection import train_test_split, StratifiedKFold, cross_validate, GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, AdaBoostClassifier, RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

RANDOM_STATE = 42

# ================================================================
# 1) Dados
# ================================================================
car_evaluation = fetch_ucirepo(id=19)
X = car_evaluation.data.features.copy()
y = car_evaluation.data.targets.iloc[:, 0].copy()  # pega a primeira e única coluna de target

categorical_cols = list(X.columns)

# Pré-processamento: One-Hot para todas as categóricas
ohe = OneHotEncoder(handle_unknown="ignore", sparse_output=True)
preprocess = ColumnTransformer(
    transformers=[("cat", ohe, categorical_cols)],
    remainder="drop"
)

# ================================================================
# 2) Split Holdout (80/20) + CV no treino
# ================================================================
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=RANDOM_STATE, stratify=y
)

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)
scoring = {"acc": "accuracy", "f1m": "f1_macro"}  # usaremos f1_macro para seleção

# ================================================================
# 3) Modelos base
#    Obs.: class_weight="balanced" ajuda com desbalanceamento
# ================================================================
models = {
    "DecisionTree": DecisionTreeClassifier(
        criterion="gini",
        max_depth=None,
        min_samples_leaf=1,
        class_weight="balanced",
        random_state=RANDOM_STATE
    ),
    "Bagging(Tree)": BaggingClassifier(
        estimator=DecisionTreeClassifier(
            criterion="gini",
            max_depth=None,
            min_samples_leaf=1,
            class_weight="balanced",
            random_state=RANDOM_STATE
        ),
        n_estimators=200,
        max_samples=1.0,
        max_features=1.0,
        bootstrap=True,
        n_jobs=-1,
        random_state=RANDOM_STATE
    ),
    "AdaBoost": AdaBoostClassifier(
        # stump por default
        n_estimators=300,
        learning_rate=0.5,
        random_state=RANDOM_STATE
    ),
    "RandomForest": RandomForestClassifier(
        n_estimators=400,
        max_depth=None,
        min_samples_leaf=1,
        class_weight="balanced",
        n_jobs=-1,
        random_state=RANDOM_STATE
    )
}

# ================================================================
# 4) Avaliação por CV no treino e seleção do melhor por F1 macro
# ================================================================
summary_rows = []
pipes = {}
for name, clf in models.items():
    pipe = Pipeline(steps=[("prep", preprocess), ("clf", clf)])
    pipes[name] = pipe

    cv_results = cross_validate(
        pipe, X_train, y_train,
        scoring=scoring, cv=cv, n_jobs=-1
    )

    row = {
        "model": name,
        "cv_acc_mean": cv_results["test_acc"].mean(),
        "cv_acc_std":  cv_results["test_acc"].std(ddof=1),
        "cv_f1m_mean": cv_results["test_f1m"].mean(),
        "cv_f1m_std":  cv_results["test_f1m"].std(ddof=1),
    }
    summary_rows.append(row)

summary_df = pd.DataFrame(summary_rows).sort_values("cv_f1m_mean", ascending=False)
best_model_name = summary_df.iloc[0]["model"]
print("\n==================== RESUMO CV (5-fold, treino) ====================")
print(summary_df.to_string(index=False, float_format=lambda x: f"{x:0.4f}"))
print(f"\n>> Selecionado para tuning (maior F1 macro médio): {best_model_name}")

# ================================================================
# 5) Tuning do melhor modelo com GridSearchCV
#    - grades específicas para cada algoritmo
#    - usamos F1 macro como métrica de otimização
# ================================================================
param_grids = {
    "DecisionTree": {
        "clf__criterion": ["gini", "entropy", "log_loss"],
        "clf__max_depth": [None, 5, 8, 12, 16],
        "clf__min_samples_leaf": [1, 2, 4, 8, 12]
    },
    "Bagging(Tree)": {
        "clf__n_estimators": [100, 200, 400],
        "clf__max_samples": [0.6, 0.8, 1.0],
        "clf__max_features": [0.6, 0.8, 1.0],
        # hiper da árvore base:
        "clf__estimator__max_depth": [None, 4, 6, 10],
        "clf__estimator__min_samples_leaf": [1, 2, 4]
    },
    "AdaBoost": {
        "clf__n_estimators": [100, 200, 300, 500],
        "clf__learning_rate": [0.1, 0.3, 0.5, 1.0]
        # (opcional) base_estimator raso:
        # "clf__estimator": [DecisionTreeClassifier(max_depth=1, random_state=RANDOM_STATE)]
    },
    "RandomForest": {
        "clf__n_estimators": [200, 400, 800],
        "clf__max_depth": [None, 8, 12, 16],
        "clf__min_samples_leaf": [1, 2, 4],
        "clf__max_features": ["sqrt", "log2", None]
    }
}

best_base_pipe = pipes[best_model_name]
grid = GridSearchCV(
    estimator=best_base_pipe,
    param_grid=param_grids[best_model_name],
    scoring="f1_macro",
    cv=cv,
    n_jobs=-1,
    refit=True,
    verbose=0
)

grid.fit(X_train, y_train)

print("\n==================== MELHOR CONJUNTO (GridSearchCV) ====================")
print(f"Algoritmo: {best_model_name}")
print(f"Melhor F1_macro (CV): {grid.best_score_:0.4f}")
print("Melhores hiperparâmetros:")
for k, v in grid.best_params_.items():
    print(f"  {k}: {v}")

# ================================================================
# 6) Comparação em TESTE (holdout 20%): antes vs. depois do tuning
# ================================================================
# 6.1 modelo base (sem tuning)
base_pipe = best_base_pipe.fit(X_train, y_train)
y_pred_base = base_pipe.predict(X_test)

print("\n==================== TESTE - MODELO BASE (sem tuning) ====================")
print(classification_report(y_test, y_pred_base, digits=4))
print("Matriz de confusão (linhas = verdade; colunas = predito):")
print(confusion_matrix(y_test, y_pred_base, labels=np.unique(y)))

# 6.2 melhor modelo tunado
best_tuned_pipe = grid.best_estimator_
y_pred_tuned = best_tuned_pipe.predict(X_test)

print("\n==================== TESTE - MODELO TUNADO (com GridSearch) ====================")
print(classification_report(y_test, y_pred_tuned, digits=4))
print("Matriz de confusão (linhas = verdade; colunas = predito):")
print(confusion_matrix(y_test, y_pred_tuned, labels=np.unique(y)))

# 6.3 relatório resumido lado a lado (F1 macro e Acurácia)
from sklearn.metrics import f1_score, accuracy_score
def summarize_test(y_true, y_pred):
    return {
        "acc_test": accuracy_score(y_true, y_pred),
        "f1_macro_test": f1_score(y_true, y_pred, average="macro")
    }

res_base = summarize_test(y_test, y_pred_base)
res_tuned = summarize_test(y_test, y_pred_tuned)

print("\n==================== COMPARAÇÃO FINAL (TESTE) ====================")
print(pd.DataFrame(
    {
        "modelo": ["Base - " + best_model_name, "Tuned - " + best_model_name],
        "acc_test": [f"{res_base['acc_test']:.4f}", f"{res_tuned['acc_test']:.4f}"],
        "f1_macro_test": [f"{res_base['f1_macro_test']:.4f}", f"{res_tuned['f1_macro_test']:.4f}"]
    }
).to_string(index=False))



==================== RESUMO CV (5-fold, treino) ====================
        model  cv_acc_mean  cv_acc_std  cv_f1m_mean  cv_f1m_std
Bagging(Tree)       0.9718      0.0119       0.9495      0.0136
 DecisionTree       0.9559      0.0113       0.9347      0.0160
 RandomForest       0.9522      0.0101       0.8830      0.0239
     AdaBoost       0.8502      0.0170       0.4803      0.0520

>> Selecionado para tuning (maior F1 macro médio): Bagging(Tree)

==================== MELHOR CONJUNTO (GridSearchCV) ====================
Algoritmo: Bagging(Tree)
Melhor F1_macro (CV): 0.9503
Melhores hiperparâmetros:
  clf__estimator__max_depth: 10
  clf__estimator__min_samples_leaf: 1
  clf__max_features: 1.0
  clf__max_samples: 0.8
  clf__n_estimators: 200

==================== TESTE - MODELO BASE (sem tuning) ====================
              precision    recall  f1-score   support

         acc     0.9733    0.9481    0.9605        77
        good     0.8750    1.0000    0.9333        14
       

# 3º Pipeline -- OVO e OVR

In [18]:
# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd

from ucimlrepo import fetch_ucirepo
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_validate
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.tree import DecisionTreeClassifier
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier

from sklearn.metrics import classification_report, confusion_matrix, f1_score, accuracy_score

RANDOM_STATE = 42

# ================================================================
# 1) Dados
# ================================================================
car = fetch_ucirepo(id=19)
X = car.data.features.copy()
y = car.data.targets.iloc[:, 0].copy()

categorical_cols = list(X.columns)
ohe = OneHotEncoder(handle_unknown="ignore", sparse_output=True)
prep = ColumnTransformer([("cat", ohe, categorical_cols)], remainder="drop")

# ================================================================
# 2) Holdout + CV
# ================================================================
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, stratify=y, random_state=RANDOM_STATE
)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)
scoring = {"acc": "accuracy", "f1m": "f1_macro"}

# ================================================================
# 3) Classificador base (árvore) e estratégias OvR / OvO
# ================================================================
base_tree = DecisionTreeClassifier(
    criterion="gini",
    max_depth=None,
    min_samples_leaf=1,
    class_weight="balanced",
    random_state=RANDOM_STATE
)

ovr = OneVsRestClassifier(base_tree)
ovo = OneVsOneClassifier(base_tree)

pipelines = {
    "OvR (One-vs-Rest)": Pipeline([("prep", prep), ("clf", ovr)]),
    "OvO (One-vs-One)": Pipeline([("prep", prep), ("clf", ovo)])
}

# ================================================================
# 4) Avaliação por CV no treino
# ================================================================
rows = []
for name, pipe in pipelines.items():
    cv_res = cross_validate(pipe, X_train, y_train, scoring=scoring, cv=cv, n_jobs=-1)
    rows.append({
        "strategy": name,
        "cv_acc_mean": cv_res["test_acc"].mean(),
        "cv_acc_std":  cv_res["test_acc"].std(ddof=1),
        "cv_f1m_mean": cv_res["test_f1m"].mean(),
        "cv_f1m_std":  cv_res["test_f1m"].std(ddof=1),
    })

cv_df = pd.DataFrame(rows).sort_values("cv_f1m_mean", ascending=False)
print("\n==================== RESUMO CV (5-fold, treino) ====================")
print(cv_df.to_string(index=False, float_format=lambda x: f"{x:0.4f}"))

best_name = cv_df.iloc[0]["strategy"]
print(f"\n>> Selecionado (maior F1 macro médio): {best_name}")

# ================================================================
# 5) Avaliação em TESTE (holdout 20%)
# ================================================================
def eval_on_test(name, pipe):
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)
    print(f"\n==================== TESTE — {name} ====================")
    print(classification_report(y_test, y_pred, digits=4))
    print("Matriz de confusão (linhas = verdade; colunas = predito):")
    print(confusion_matrix(y_test, y_pred, labels=np.unique(y)))
    return {
        "model": name,
        "acc_test": accuracy_score(y_test, y_pred),
        "f1_macro_test": f1_score(y_test, y_pred, average="macro")
    }

test_rows = []
for name, pipe in pipelines.items():
    test_rows.append(eval_on_test(name, pipe))

test_df = pd.DataFrame(test_rows).sort_values("f1_macro_test", ascending=False)
print("\n==================== COMPARAÇÃO FINAL (TESTE) ====================")
print(test_df.to_string(index=False, float_format=lambda x: f"{x:0.4f}"))

winner = test_df.iloc[0]["model"]
print(f"\n>>> MELHOR EM TESTE (por F1 macro): {winner}")



==================== RESUMO CV (5-fold, treino) ====================
         strategy  cv_acc_mean  cv_acc_std  cv_f1m_mean  cv_f1m_std
 OvO (One-vs-One)       0.9761      0.0094       0.9534      0.0339
OvR (One-vs-Rest)       0.9501      0.0199       0.8251      0.0774

>> Selecionado (maior F1 macro médio): OvO (One-vs-One)

==================== TESTE — OvR (One-vs-Rest) ====================
              precision    recall  f1-score   support

         acc     0.9853    0.8701    0.9241        77
        good     0.9231    0.8571    0.8889        14
       unacc     0.9918    1.0000    0.9959       242
       vgood     0.5714    0.9231    0.7059        13

    accuracy                         0.9624       346
   macro avg     0.8679    0.9126    0.8787       346
weighted avg     0.9718    0.9624    0.9647       346

Matriz de confusão (linhas = verdade; colunas = predito):
[[ 67   1   2   7]
 [  0  12   0   2]
 [  0   0 242   0]
 [  1   0   0  12]]

==================== TESTE — 

# Análise + Discussão

**Proposta:** Comparar OvR vs OvO com Árvore de Decisão no dataset Car Evaluation

Eu fiz um experimento para transformar um problema com 4 classes em vários problemas binários, usando duas estratégias diferentes:

* **OvR (One-vs-Rest):** uma classe contra todas as outras.
* **OvO (One-vs-One):** disputa entre cada par de classes.

**Árvore de Decisão** como modelo base nas duas estratégias.

---

### Procedimento

1. Separei os dados em **treino (80%)** e **teste (20%)**.
2. No **treino**, fiz **validação cruzada com 5 partes** para ter uma avaliação mais estável.
3. A métrica principal que escolhida foi **F1 macro**, porque ela dá o mesmo peso para cada classe, uma vez que os data set é desbalanceado. Também olhei a **acurácia** para comparar.

---

### Resultados

**No treino (validação cruzada):**

* **OvO**: F1 macro ≈ **0,953** (média)
* **OvR**: F1 macro ≈ **0,825** (média)
  → **OvO** foi bem melhor e também mais estável (variação menor).

**No teste:**

* **OvO**: F1 macro = **0,964** e acurácia = **0,986**
* **OvR**: F1 macro = **0,879** e acurácia = **0,962**
  → **OvO** continuou vencendo com folga.

---

### Análise

* O **OvO** acerta melhor **todas** as classes de forma equilibrada, inclusive as **menos comuns** (como *good* e *vgood*).
* O **OvR** teve mais dificuldade, principalmente porque “uma classe contra o resto” junta muitos casos diferentes no grupo “resto”, e a árvore tem mais chance de se confundir, gerando **falsos positivos** (por exemplo, chamando “vgood” quando não era).

---

### Onde o OvO se destacou

* Para a classe **vgood** (que tem poucos exemplos), o **OvO** quase não errou: **lembrou de todos** (recall 100%) e quase não “inventou” vgood quando não era (alta precisão).
* Para a classe **acc**, o **OvO** também errou menos, evitando confundir com **vgood**.

---

### Por que o OvO tende a ser melhor aqui

* No **OvR**, o “resto” é um grupo **grande e misturado**, o que deixa a fronteira de decisão confusa.
* No **OvO**, cada disputa é **só entre duas classes**, então a árvore consegue separar melhor, porque o problema fica **mais simples** em cada duelo.

---

### Conclusão

* **Vencedor:** **OvO (One-vs-One) com Árvore de Decisão**.
* **Motivo:** entregou **F1 macro maior** tanto no treino quanto no teste, mostrando que trata **todas as classes** de forma mais justa e com menos confusão.
